# Testing BAML Logging in Jupyter

This notebook demonstrates how to capture BAML logs in Jupyter notebook output.

In [ ]:
# First, set up the environment
import os
import sys
import subprocess
import logging

# Set BAML_LOG environment variable
os.environ['BAML_LOG'] = 'info'
print(f"BAML_LOG is set to: {os.environ['BAML_LOG']}")

In [ ]:
# Method 1: Configure Python logging to capture subprocess output
import logging

# Configure logging for Jupyter - force reconfiguration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    force=True,  # This forces reconfiguration in Jupyter
    handlers=[
        logging.StreamHandler(sys.stdout)  # Output to stdout instead of stderr
    ]
)

# Get logger for BAML
logger = logging.getLogger('baml')
logger.setLevel(logging.DEBUG)

In [ ]:
# Method 2: Monkey-patch print to capture BAML output
import builtins
from datetime import datetime

# Store original print function
_original_print = builtins.print

def patched_print(*args, **kwargs):
    """Patched print function that adds timestamp and formatting."""
    # Check if this is a BAML log message
    message = ' '.join(str(arg) for arg in args)
    if '[baml' in message.lower() or 'baml' in message.lower():
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        _original_print(f"[{timestamp}] BAML LOG: {message}", **kwargs)
    else:
        _original_print(*args, **kwargs)

# Apply the patch
builtins.print = patched_print

In [ ]:
# Method 3: Use IPython's capture magic
from IPython.utils.capture import capture_output

def run_with_capture(func, *args, **kwargs):
    """Run a function and capture all its output."""
    with capture_output() as captured:
        result = func(*args, **kwargs)
    
    # Display captured output
    if captured.stdout:
        print("=== Standard Output ===")
        print(captured.stdout)
    
    if captured.stderr:
        print("\n=== Error/Log Output ===")
        print(captured.stderr)
    
    # Also display using IPython's display
    from IPython.display import display, HTML
    if captured.stderr:
        display(HTML(f"<pre style='background-color: #f0f0f0; padding: 10px;'>{captured.stderr}</pre>"))
    
    return result

In [ ]:
# Method 4: Direct stderr/stdout redirection
import io
import contextlib

@contextlib.contextmanager
def capture_all_output():
    """Capture both stdout and stderr."""
    old_stdout = sys.stdout
    old_stderr = sys.stderr
    
    stdout_buffer = io.StringIO()
    stderr_buffer = io.StringIO()
    
    # Create a custom writer that writes to both the buffer and the notebook
    class TeeWriter:
        def __init__(self, buffer, original):
            self.buffer = buffer
            self.original = original
        
        def write(self, data):
            self.buffer.write(data)
            self.original.write(data)
            self.original.flush()  # Ensure immediate display
        
        def flush(self):
            self.buffer.flush()
            self.original.flush()
    
    try:
        sys.stdout = TeeWriter(stdout_buffer, old_stdout)
        sys.stderr = TeeWriter(stderr_buffer, old_stderr)
        yield stdout_buffer, stderr_buffer
    finally:
        sys.stdout = old_stdout
        sys.stderr = old_stderr

In [ ]:
# Test function that would use BAML
def test_baml_function():
    print("Starting BAML test...")
    # This would be where your main() function runs
    # For now, let's simulate some BAML-like output
    print("[BAML] Initializing client...")
    print("[BAML] Running DetermineNextStep...")
    print("[BAML] Reasoning: User wants to multiply 3 and 4")
    print("[BAML] Result: MultiplyTool(a=3, b=4)")
    return "Test completed"

In [ ]:
# Test different methods
print("Testing BAML logging capture methods...\n")

# Method 1: Using IPython capture
print("Method 1: IPython capture")
result = run_with_capture(test_baml_function)
print(f"Result: {result}\n")

In [ ]:
# Method 2: Using context manager
print("Method 2: Direct output capture")
with capture_all_output() as (stdout, stderr):
    result = test_baml_function()
print(f"Result: {result}")

## For actual BAML usage

When using actual BAML functions, the logs should appear if:

1. `BAML_LOG` environment variable is set (we set it to 'info')
2. You're using one of the capture methods above

Example with real BAML:

```python
# Assuming you have your main() function from the workshop
os.environ['BAML_LOG'] = 'info'

# Use IPython capture
with capture_output() as captured:
    main("can you multiply 3 and 4")

# Display the logs
print("=== Output ===")
print(captured.stdout)
print("\n=== Logs ===")
print(captured.stderr)
```

In [ ]:
# Method 5: Enable verbose subprocess output
# This is useful if BAML is running as a subprocess

def run_baml_with_logging():
    # Ensure BAML_LOG is set
    env = os.environ.copy()
    env['BAML_LOG'] = 'info'
    
    # If BAML runs as subprocess, capture its output
    result = subprocess.run(
        ['python', '-c', 'print("BAML subprocess test")'],
        env=env,
        capture_output=True,
        text=True
    )
    
    print("Subprocess stdout:", result.stdout)
    print("Subprocess stderr:", result.stderr)

run_baml_with_logging()